In [ ]:
class Timer:  
    """
    记录多次运行时间
    示例:
    c = torch.zeros(n)
    timer = Timer()
    for i in range(n):
        c[i] = a[i] + b[i]
    f'{timer.stop():.5f} sec'
    """
    def __init__(self):
        self.times = []
        self.start()

    def start(self):
        """启动计时器"""
        self.tik = time.time()

    def stop(self):
        """停止计时器并将时间记录在列表中"""
        self.times.append(time.time() - self.tik)
        return self.times[-1]

    def avg(self):
        """返回平均时间"""
        return sum(self.times) / len(self.times)

    def sum(self):
        """返回时间总和"""
        return sum(self.times)

    def cumsum(self):
        """返回累计时间"""
        return np.array(self.times).cumsum().tolist()

In [ ]:
def accuracy(y_hat, y):
    """
    计算预测正确的数量
    输入：
    y_hat: 预测值
    y: 真实值
    输出：
    预测正确的数量
    """
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1: # 如果预测结果是矩阵
        y_hat = y_hat.argmax(axis=1)    # one-hot转索引
    cmp = y_hat.type(y.dtype) == y      # 比较真实标签
    return float(cmp.type(y.float).sum())

In [5]:
def evaluate_accuracy(net, data_iter):
    """
    计算模型在数据集上的准确率
    输入：
    net: 神经网络
    data_iter: 数据集
    输出：
    准确率
    """
    if isinstance(net, torch.nn.Module):
        net.eval() # 设置为评估模式
    metric = Accumulator(2) # 正确预测数，预测总数
    for X, y in data_iter:
        metric.add(acciracy(net(X), y), y.numel())
    return metric[0] / metric[1]

In [4]:
class Accumulator:
    """
    在多个变量上累加
    输入：
    n: 变量数量
    *args: 变量
    idx: 变量索引
    输出:
    索引对应的变量
    """
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [7]:
def train_epoch_ch3(net, train_iter, loss, updater):
    """
    训练模型一个迭代周期
    输入：
    net: 神经网络
    train_iter: 训练数据集
    loss: 损失函数
    updater: 更新参数的函数
    输出：
    训练损失、训练准确度
    """
    # 将模型设置为训练模式
    if isinstance(net, torch.nn.Module):
        net.train()
    # 训练损失总和、训练准确度总和、样本数
    metric = Accumulator(3)
    for X, y in train_iter:
        # 计算梯度并更新参数
        y_hat = net(X)
        l = loss(y_hat, y)
        if isinstance(updater, torch.optim.Optimizer):
            # 使用PyTorch内置的优化器和损失函数
            updater.zero_grad()
            l.mean().backward()
            updater.step()
        else:
            # 使用定制的优化器和损失函数
            l.sum().backward()
            updater(X.shape[0])
        metric.add(float(l.sum()), accuracy(y_hat, y), y.numel())
    # 返回训练损失和训练精度
    return metric[0] / metric[2], metric[1] / metric[2]

In [ ]:
from IPython import display
from matplotlib import pyplot as plt


def use_svg_display():
    """使用 svg 格式在 Jupyter 中显示绘图"""
    display.set_matplotlib_formats('svg')



def set_axes(axes, xlabel, ylabel, xlim, ylim, xscale='linear', yscale='linear', legend=None):
    """设置 matplotlib 的坐标轴。"""
    axes.set_xlabel(xlabel)
    axes.set_ylabel(ylabel)
    axes.set_xscale(xscale)
    axes.set_yscale(yscale)
    axes.set_xlim(xlim)
    axes.set_ylim(ylim)
    if legend:
        axes.legend(legend)
    axes.grid()

class Animator:
    """
    在动画中动态绘制数据
    输入：
    xlabel: x轴标签
    ylabel: y轴标签
    legend: 图例
    xlim: x轴范围
    ylim: y轴范围
    xscale: x轴标度
    yscale: y轴标度
    fmts: 格式
    nrows: 子图行数
    ncols: 子图列数
    figsize: 图表大小
    """
    def __init__(self, xlabel=None, ylabel=None, legend=None, xlim=None,
                 ylim=None, xscale='linear', yscale='linear',
                 fmts=('-', 'm--', 'g-.', 'r:'), nrows=1, ncols=1,
                 figsize=(3.5, 2.5)):
        # 增量地绘制多条线
        if legend is None:
            legend = []
        use_svg_display()
        self.fig, self.axes = plt.subplots(nrows, ncols, figsize=figsize)
        if nrows * ncols == 1:
            self.axes = [self.axes, ]
        # 使用lambda函数捕获参数
        self.config_axes = lambda: set_axes(
            self.axes[0], xlabel, ylabel, xlim, ylim, xscale, yscale, legend)
        self.X, self.Y, self.fmts = None, None, fmts

    def add(self, x, y):
        # 向图表中添加多个数据点
        if not hasattr(y, "__len__"):
            y = [y]
        n = len(y)
        if not hasattr(x, "__len__"):
            x = [x] * n
        if not self.X:
            self.X = [[] for _ in range(n)]
        if not self.Y:
            self.Y = [[] for _ in range(n)]
        for i, (a, b) in enumerate(zip(x, y)):
            if a is not None and b is not None:
                self.X[i].append(a)
                self.Y[i].append(b)
        self.axes[0].cla()
        for x, y, fmt in zip(self.X, self.Y, self.fmts):
            self.axes[0].plot(x, y, fmt)
        self.config_axes()
        display.display(self.fig)
        display.clear_output(wait=True)

In [ ]:
def train_ch3(net, train_iter, test_iter, loss, num_epochs, updater):
    """
    训练模型函数
    输入：
    net: 神经网络
    train_iter: 训练数据集
    test_iter: 测试数据集
    loss: 损失函数
    num_epochs: 迭代周期数
    updater: 更新参数的函数
    输出：
    画图
    """
    animator = Animator(xlabel='epoch', xlim=[1, num_epochs], ylim=[0.3, 0.9],
                        legend=['train loss', 'train acc', 'test acc'])
    for epoch in range(num_epochs):
        train_metrics = train_epoch_ch3(net, train_iter, loss, updater)
        test_acc = evaluate_accuracy(net, test_iter)
        animator.add(epoch + 1, train_metrics + (test_acc,))
    train_loss, train_acc = train_metrics
    assert train_loss < 0.5, train_loss
    assert train_acc <= 1 and train_acc > 0.7, train_acc
    assert test_acc <= 1 and test_acc > 0.7, test_acc

In [ ]:
def predict_ch3(net, test_iter, n=6):
    """
    预测函数
    输入：
    net: 神经网络
    test_iter: 测试数据集
    n: 预测样本数
    输出：
    画图
    """
    for X, y in test_iter:
        break
    trues = d2l.get_fashion_mnist_labels(y)
    preds = d2l.get_fashion_mnist_labels(net(X).argmax(axis=1))
    titles = [true + '\n' + pred for true, pred in zip(trues, preds)]
    d2l.show_images(
        X[0:n].reshape((n, 28, 28)), 1, n, titles=titles[0:n])

In [ ]:
def show_images(imgs, num_rows, num_cols, titles=None, scale=1.5):  
    """
    绘制图像列表
    输入:
    imgs: 图像列表
    num_rows: 行数
    num_cols: 列数
    titles: 标题列表
    scale: 图片缩放比例
    """
    figsize = (num_cols * scale, num_rows * scale)
    _, axes = d2l.plt.subplots(num_rows, num_cols, figsize=figsize)
    axes = axes.flatten()
    for i, (ax, img) in enumerate(zip(axes, imgs)):
        if torch.is_tensor(img):
            # 图片张量
            ax.imshow(img.numpy())
        else:
            # PIL图片
            ax.imshow(img)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        if titles:
            ax.set_title(titles[i])
    return axes